# JobMatchCV: AI-Powered Resume Optimizer

This project leverages the AutoGen framework to create a sophisticated multi-agent system for enhancing CVs based on specific job advertisements. The system now includes automatic conversion to a polished markdown format for the final output.

## Key Agents:

- **User Input Agent**: Handles initial user input (CV file and job ad link) and manages text extraction.
- **CV Analysis Agent**: Examines the structure, content, and formatting of the provided CV.
- **Job Analysis Agent**: Dissects job requirements, qualifications, and desired skills from the advertisement.
- **ATS Standards Agent**: Provides expertise on Applicant Tracking System (ATS) standards and best practices.
- **CV Enhancement Agent**: Compares CV content with job requirements and generates improvement suggestions.
- **User Output Agent**: Presents enhancement suggestions in a clear, actionable format.
- **Markdown Conversion Agent**: Transforms the enhanced CV into a well-structured markdown document.

## Process Flow:

1. Extract text from the user's CV (PDF or DOCX format)
2. Fetch and parse the job description from the provided link
3. Analyze both the CV and job requirements
4. Generate tailored enhancement suggestions
5. Convert the improved CV to a professional markdown format
6. Save the result as a `cv.md` file

## Key Features:

- Multi-agent collaboration for comprehensive CV analysis and enhancement
- Support for PDF and DOCX input formats
- Web scraping capability for job description extraction
- ATS-compliant suggestions
- Automatic markdown conversion for the final CV
- File saving functionality for easy user access to the enhanced CV

This system streamlines the process of tailoring a CV to a specific job opportunity, ensuring that the final product is both content-optimized and formatted for maximum impact.

## Step 1: Install Dependencies

In [ ]:
!pip install openai pyautogen requests beautifulsoup4 PyPDF2 python-docx python-dotenv
!pip install mistralai
!pip install pyautogen[google]
!pip install pyautogen["anthropic"]
!pip install pyautogen[mistral]

## Step 2: Load Environment Variables
Set your API keys and Autogen configuration:

In [ ]:
import os
import autogen
from google.auth.exceptions import DefaultCredentialsError
from google.oauth2 import service_account
from typing import Dict, Any



# Initialize the configuration list for GPT-4
config_list_gpt4 = [
    {
        "model": "gpt-4o",
        "api_key": "your-api-key",
        "api_type": "openai"
    }
]

# Configuration for Gemini 1.5 Pro
config_list_gemini = [
    {
        "model": "gemini-1.5-pro-2m-latest",
        "api_key": "your-api-key",
        "api_type": "google"
    }
]

# Configuration for Claude 3.5 Sonnet
config_list_claude = [
    {
        "model": "claude-3-5-sonnet-20240620",
        "api_key": "your-api-key",
        "api_type": "anthropic"
    }
]

# Configuration for Mistral
config_list_mistral = [
    {
        "model": "mistral-large-latest",
        "api_key": "your-api-key",
        "api_type": "mistral"
    }
]

## Step 3: Define Utility Functions
Define the utility functions for text extraction and web scraping:

In [ ]:
import requests
from bs4 import BeautifulSoup
from PyPDF2 import PdfReader
from docx import Document
from autogen import register_function

def fetch_job_description(job_link: str) -> str:
    response = requests.get(job_link)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        job_description = soup.get_text()
        return job_description
    else:
        raise ValueError(f"Failed to fetch the job description. HTTP status code: {response.status_code}")

def extract_cv_text(file_path: str) -> str:
    file_ext = file_path.split(".")[-1].lower()
    if file_ext == "pdf":
        return extract_text_from_pdf(file_path)
    elif file_ext == "docx":
        return extract_text_from_docx(file_path)
    else:
        raise ValueError("Unsupported file format. Please provide a PDF or DOCX file.")

def extract_text_from_pdf(file_path: str) -> str:
    with open(file_path, "rb") as file:
        reader = PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
        return text

def extract_text_from_docx(file_path: str) -> str:
    doc = Document(file_path)
    return "\n".join([para.text for para in doc.paragraphs])

def save_markdown_cv(content: str, filename: str = "cv.md") -> str:
    with open(filename, "w", encoding="utf-8") as file:
        file.write(content)
    return f"CV saved as {filename}"

## Step 4: Define and Initialize Agents

In this step, we define and initialize the agents that will collaborate to enhance your CV. You can customize the agent configuration based on your preferences and available API keys.

### Default Configuration:
- **GPT-4o**: Used for receiving input and analyzing CV and job descriptions.
- **Claude 3.5 Sonnet**: Employed for CV editing and enhancement due to its strong performance in text generation and refinement tasks.

Feel free to experiment with different combinations of agents to find the optimal setup for your needs. The provided code includes configurations for multiple models, including GPT-4o, Gemini, Claude, and Mistral.


In [ ]:
def termination_msg(x):
    return isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()

llm_config_gpt4 = {
    "cache_seed": 42,
    "temperature": 0,
    "config_list": config_list_gpt4,
    "timeout": 120,
}

llm_config_gemini = {
    "cache_seed": 42,
    "temperature": 0,
    "config_list": config_list_gemini,
    "timeout": 120,
}

llm_config_claude = {
    "cache_seed": 42,
    "temperature": 0,
    "config_list": config_list_claude,
    "timeout": 120,
}

llm_config_mistral = {
    "cache_seed": 42,
    "temperature": 0.5,
    "max_tokens": 10000,
    "safe_prompt": False,
    "config_list": config_list_mistral,
    "timeout": 120,
}

# User Input Agent
user_input_agent = autogen.UserProxyAgent(
    name="UserInputAgent",
    is_termination_msg=termination_msg,
    llm_config=llm_config_gpt4,
    system_message= """
You are a User Input Agent. Your role is to receive extract_text_from_docx or extract_text_from_pdf, along with job_description,
and pass both the extracted CV text and the job description to the CV Analysis Agent and Job Analysis Agent.
Reply 'TERMINATE' when everything is done.
""",
    function_map={
        "fetch_job_description": fetch_job_description,
        "extract_cv_text": extract_cv_text,
    }
)
cv_analysis_agent = autogen.AssistantAgent(
    name="CVAnalysisAgent",
    is_termination_msg=termination_msg,
    llm_config=llm_config_gpt4,
    system_message="""
You are a CV Analysis Agent. Your task is to analyze the structure, content,
and formatting of the user's CV. Identify key sections like summary, work experience, education, and skills.
Structure this data in a format that can be easily consumed by other agents.
Pass the analyzed CV data to the CV Enhancement Agent.
""",
)

job_analysis_agent = autogen.AssistantAgent(
    name="JobAnalysisAgent",
    is_termination_msg=termination_msg,
    llm_config=llm_config_gpt4,
    system_message="""
You are a Job Analysis Agent. Your task is to analyze job requirements,
qualifications, and desired skills from the job description.
Extract important keywords, phrases, and requirements related to the job.
Structure the analyzed job data in a format that can be easily consumed by the CV Enhancement Agent.
""",
)

ats_standards_agent = autogen.AssistantAgent(
    name="ATSStandardsAgent",
    is_termination_msg=termination_msg,
    llm_config=llm_config_gemini,
    system_message="""
You are an ATS Standards Agent. Your task is to provide knowledge and guidelines
about Applicant Tracking Systems (ATS), including CV formatting and keyword optimization best practices.
Provide clear and actionable guidelines to ensure compliance with ATS.
""",
)

cv_enhancement_agent = autogen.AssistantAgent(
    name="CVEnhancementAgent",
    is_termination_msg=termination_msg,
    llm_config=llm_config_gpt4,
    system_message="""
You are a CV Enhancement Agent. Your task is to compare the CV content with the job requirements
and ATS guidelines, identify areas for improvement, and generate specific suggestions for modifications.
Ensure your suggestions are clear and actionable.
""",
)

markdown_conversion_agent = autogen.AssistantAgent(
    name="MarkdownConversionAgent",
    is_termination_msg=termination_msg,
    llm_config=llm_config_gpt4,
    system_message="""
You are a Markdown Conversion Agent. Your task is to convert the enhanced CV content into a well-formatted markdown document.
Ensure proper use of markdown syntax for headings, lists, and emphasis. Structure the CV sections clearly and professionally.
Reply 'TERMINATE' when everything is done.
"""
)

user_output_agent = autogen.AssistantAgent(
    name="UserOutputAgent",
    is_termination_msg=termination_msg,
    llm_config=llm_config_gemini,
    system_message="""
You are a User Output Agent. Your task is to present CV enhancement suggestions in a clear and organized manner.
Specify the exact changes and modifications needed for each part of the CV.
Ensure the suggestions are user-friendly. Reply 'TERMINATE' when everything is done.
""",
)

register_function(
    fetch_job_description,
    caller=user_input_agent,
    executor=cv_analysis_agent,
    name="fetch_job_description",
    description="Job description fetch",
)
register_function(
    extract_cv_text,
    caller=user_input_agent,
    executor=cv_analysis_agent,
    name="extract_cv_text",
    description="Text extracted from CV",
)
register_function(
    extract_text_from_pdf,
    caller=user_input_agent,
    executor=cv_analysis_agent,
    name="extract_text_from_pdf",
    description="Text extracted from PDF",
)
register_function(
    extract_text_from_docx,
    caller=user_input_agent,
    executor=cv_analysis_agent,
    name="extract_text_from_docx",
    description="Text extrcated from word file",
)

register_function(
    save_markdown_cv,
    caller=markdown_conversion_agent,
    executor=user_input_agent,
    name="save_markdown_cv",
    description="Save the markdown CV content to a file named CV.md",
)

## Step 5: User Input and Processing
This step allows users to upload their CV and provide a job advertisement link for analysis.

In [ ]:
from google.colab import files
import os

# Function to upload CV file
def upload_cv():
    print("Please upload your CV (PDF or DOCX format):")
    uploaded = files.upload()
    if uploaded:
        filename = list(uploaded.keys())[0]
        return os.path.join('/content', filename)
    else:
        raise ValueError("No file was uploaded.")

# Get CV file path from user
cv_path = upload_cv()

# Get job advertisement link from user
job_ad_link = input("Please enter the job advertisement link: ")

# Process user input and get initial CV content and job description
cv_content = extract_cv_text(cv_path)
job_description = fetch_job_description(job_ad_link)

## Step 6: Group Chat Setup and Enhancement Process
Here, we set up the group chat with all defined agents and initiate the CV enhancement process.

In [ ]:
group_chat = autogen.GroupChat(
    agents=[user_input_agent, cv_analysis_agent, job_analysis_agent,
            ats_standards_agent, cv_enhancement_agent, user_output_agent,
            markdown_conversion_agent],
    messages=[],
    speaker_selection_method="auto",
    max_round=15
)


# Create a GroupChatManager
manager = autogen.GroupChatManager(groupchat=group_chat, llm_config=llm_config_gpt4)


# Reset the agents to ensure a clean state
for agent in group_chat.agents:
    agent.reset()

# Function to initiate interaction with the User Input Agent
def initiate_cv_enhancement(cv_content: str, job_description: str):
    initial_message = f"""
    CV Content: {cv_content}
    Job Description: {job_description}

    Please analyze the CV and job description, enhance the CV accordingly, and then convert the enhanced CV to a markdown format.
    Finally, save the markdown CV using the save_markdown_cv function.
    """
    user_input_agent.initiate_chat(manager, message=initial_message)


user_input_agent.initiate_chat(manager, message="CV:" + cv_content + "Job Description:" + job_description )


# Start the CV enhancement process
response = initiate_cv_enhancement(cv_content, job_description)

print(response)